In [2]:
import os
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
%config IPCompleter.greedy=True

In [56]:
df = pd.read_csv("SETTINGS.csv", sep = ',')
settings1 = dict(zip(df['NAME'], df['VALUE']))
df

,NAME,VALUE
0,dyn_viscosity,1.5e-5
1,density,1.22
2,g,9.81
3,Re,2.5e+5
4,M,0.0
5,it,1000
6,ncr,9
7,alpha_min,-3
8,alpha_max,13
9,alpha_step,0.2


In [67]:
df = pd.read_csv("GEOM.csv", sep = ',')
geom1 = dict(zip(df['NAME'], df['VALUE']))


In [62]:
df = pd.read_csv("PERFORMANCE.csv", sep = ',')
performance1 = dict(zip(df['NAME'], df['VALUE']))

In [12]:
#добавить в аргументы self
def ba (geom):
    return (geom['wing_area'] / geom['AR']) ** 0.5

def Re(V, l, nu = 1.5 * 10 ** (-5)):
    return V * l / nu

def CL_required (V, S, m, settings):
    return (2 * m * 9.81 / (float(settings['density']) * V ** 2 * S))

def CDi (CL, AR, osvald_coef = 0.9):
    return (CL ** 2 / (math.pi * osvald_coef * AR))

In [27]:
def XFoil_command_CL(settings, geom, tom, V):
    command_file=open(settings['work_dir'] + 'commands.in','w')
    command_file.write('load ' + settings['work_dir'] + settings['foil1_name'] + '.dat'+'\n'\
    + settings['foil1_name'] + '\n\
    panel\n\
    oper\n\
    visc ' + str(Re(V, ba(geom))) + '\n\
    M ' + str(settings['M']) + '\n\
    type 1\n\
    vpar\n\
    n ' + str(settings['ncr']) + '\n\
    \n\
    iter\n' + str(settings['it']) + '\n\
    pacc\n'\
    + settings['work_dir'] + 'polar.dat\n\
    \n\
    cl ' + str(CL_required(V, geom['wing_area'], tom, settings)) +'\n\
    \n\
    \n\
    quit\n')
    command_file.close()
    return 0

def XFoil_run(settings):
    sep=os.path.sep
    run_xfoil_command = settings['XFoil_path'] + 'xfoil < ' + settings['work_dir'] + \
    'commands.in'
    os.system(run_xfoil_command)
    return 0

def XFoil_read(settings):
    aero_data_file = open(settings['work_dir'] + 'polar.dat', 'r')
    last_line = aero_data_file.readlines()[-1].split()
    aero_data_file.close()
    os.system('del ' + settings['work_dir'] + 'polar.dat')
    alpha, cl, cd = float(last_line[0]), float(last_line[1]), float(last_line[2])
    return [alpha, cl, cd]

def K_V_solver(geom, settings, v, AR, tom):
    polar = np.array([])
    XFoil_command_CL(settings, geom, tom, v)
    XFoil_run(settings)
    polar = np.append(polar, [[XFoil_read(settings)]])
    polar = polar.reshape(polar.size // 3, 3)
    POLAR = pd.DataFrame(polar, columns = ['alpha', 'CL', 'CD'])
    POLAR.insert(0, "AR", AR)
    POLAR.insert(1, "V", v)
    POLAR.insert(4, "CDi", CDi(POLAR.CL, AR))
    POLAR.insert(5, "K", POLAR.CL / (POLAR.CD + POLAR.CDi), )
    return POLAR

In [25]:
XFoil_command_CL(settings1, geom1, 1., 15.)
XFoil_run(settings1)
XFoil_read(settings1)

[-2.78, 0.0715, 0.0075]

In [28]:
p = K_V_solver(geom1, settings1, 15., 6., 0.8)
p

,AR,V,alpha,CL,CDi,K,CD
0,6.0,15.0,-2.91,0.0572,0.000193,7.34005,0.0076


In [38]:
def AR_selector(ar_range, geom, settings, performance, tom):
    df = pd.DataFrame()
    for ar in ar_range:
        p = K_V_solver(geom, settings, float(performance['cruise_speed']), ar, tom)
        df = pd.concat([df, p], ignore_index=True, sort=False)
    #print(df)
    return df
        

In [42]:
ar_array = np.linspace(4., 10., 25)
AR_selector(ar_array, geom1, settings1, performance1, 0.8)

,AR,V,alpha,CL,CDi,K,CD
0,4.00,26.0,-2.749,0.0761,0.000512,9.233134,0.00773
1,4.25,26.0,-2.749,0.0761,0.000482,9.267000,0.00773
2,4.50,26.0,-2.749,0.0761,0.000455,9.297313,0.00773
3,4.75,26.0,-2.749,0.0761,0.000431,9.324604,0.00773
4,5.00,26.0,-2.749,0.0761,0.000410,9.349303,0.00773
5,5.25,26.0,-2.749,0.0761,0.000390,9.371763,0.00773
6,5.50,26.0,-2.749,0.0761,0.000372,9.392274,0.00773
7,5.75,26.0,-2.749,0.0761,0.000356,9.411081,0.00773
8,6.00,26.0,-2.749,0.0761,0.000341,9.428387,0.00773
9,6.25,26.0,-2.749,0.0761,0.000328,9.444364,0.00773


In [ ]:
aero_data_file = open(settings1['work_dir'] + 'polar.dat', 'r')
lines = aero_data_file.readlines()
last_line = lines[-1].split()
aero_data_file.close()
last_line[0]